# Stop and Frisk
Brad Johnson and Steven Dye

## Problem Statement
Can we predict who will be arrested based on information obtained before the stop is initiated?

We attempt to modeling what are effective stops, i.e.,  which ones lead to arrests and  non-arrests. With the issues that surround the issue, i.e., unreasonable search and seizure being unconstitutional, having a racial bias in profiling is illegal, and police activity without any effectiveness on crime is wasted resourse, we hope to make a model that can improve the conversion rate of stops.

## Data

The data for this project was collect from a wide variety of sources, ranging from years 2003-2018. Most of it comes from the NYPD Complaint Data Historic website. This dataset includes all valid felony, misdemeanor, and violation crimes reported to the New York City Police Department (NYPD) from 2006 to the end of last year (2017). The sum of the data consists of 119 columns and 5,051,677 entires, resulting in over 1.9 GB of data.

## Data Preprocessing

The data is cleaned using the data_cleaner.py file. This consists of realligning columns so they are consistent through each year, correcting spelling mistakes, and changing entires so they are consistent as well.

## Early Data Assessment

## Modeling

Ideally we'll examine outcomes more closely. Multi-class logistic regression and random forest. You can use multiclass or holdouts. Remember that with logistic regression you drop the first dummy variable; with random forest you don't need to. Also remember to use n_jobs.

We used a balanced accuracy score.

## Future Work

* populate missing city fields from pct
* sector is less of a problem but still lots of missing values
* Gridsearch probably should be done to optimize model. Worth adding borough.